## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

In [0]:
data_I = pd.read_csv('/content/drive/My Drive/AIML_Training/Iris.csv')
data_P = pd.read_csv('/content/drive/My Drive/AIML_Training/prices.csv')



### Check all columns in the dataset

In [112]:
data_P.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [113]:
data_I.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Drop columns `date` and  `symbol`

In [0]:
data_P1 = data_P.drop(['date', 'symbol'], axis=1)

In [115]:
data_P1.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [116]:
data_P1.shape

(851264, 5)

In [117]:
data_P = data_P1.head(1000)
data_P.shape

(1000, 5)

In [118]:
data_P.dtypes

open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

In [119]:
data_P['volume'] = data_P['volume']/1000000

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [122]:
data_P.head()


,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
#X = data_P[:,0:4]
#Y = data_P[:,4]

X = data_P.drop("volume", axis=1)
Y = data_P["volume"]
# split into 70% for train and 30% for test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=7)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X_train = np.array(X_train, dtype="float32")
X_test = np.array(X_test, dtype="float32")
y_train = np.array(y_train, dtype="float32")
y_test = np.array(y_test, dtype="float32")



### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

#S=StandardScaler()
#S.fit(data_P1)
#X_train=S.fit_transform(X_train)
#X_test=S.fit_transform(X_test)

transformer = Normalizer().fit(X_train)
X_train = transformer.transform(X_train)
X_test =transformer.transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

2.Define a function to calculate prediction

In [0]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

#Normalize the data
x_n = tf.nn.l2_normalize(x,1)

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

In [0]:
#We will use normalized data
#y = tf.add(tf.matmul(x,W),b,name='output')
y = tf.add(tf.matmul(x_n,W),b,name='output')

3.Loss (Cost) Function [Mean square error]

In [0]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [140]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:X_train, y_:y_train})
    
    if epoch % 1 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)
sess.close()        

Training loss at step:  0  is  305.429
Training loss at step:  1  is  298.32846
Training loss at step:  2  is  292.8297
Training loss at step:  3  is  288.5715
Training loss at step:  4  is  285.27386
Training loss at step:  5  is  282.72018
Training loss at step:  6  is  280.74258
Training loss at step:  7  is  279.21118
Training loss at step:  8  is  278.0252
Training loss at step:  9  is  277.1068
Training loss at step:  10  is  276.39557
Training loss at step:  11  is  275.8448
Training loss at step:  12  is  275.41827
Training loss at step:  13  is  275.08798
Training loss at step:  14  is  274.83215
Training loss at step:  15  is  274.6341
Training loss at step:  16  is  274.4807
Training loss at step:  17  is  274.36194
Training loss at step:  18  is  274.2699
Training loss at step:  19  is  274.1987
Training loss at step:  20  is  274.14352
Training loss at step:  21  is  274.10077
Training loss at step:  22  is  274.06772
Training loss at step:  23  is  274.04205
Training loss

In [142]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:X_train, y_:y_train})
    
    if epoch % 5 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)
        print ('W values :' , sess.run(W))
        print ('b values :' , sess.run(b))
        
Weight_array = W.eval(session=sess)
bias_array = b.eval(session=sess)
sess.close()     

Training loss at step:  0  is  305.429
W values : [[0.16808607]
 [0.16866677]
 [0.16635722]
 [0.17008267]]
b values : [0.33661488]
Training loss at step:  5  is  282.72018
W values : [[0.7502235 ]
 [0.7528154 ]
 [0.74250716]
 [0.7591351 ]]
b values : [1.5024234]
Training loss at step:  10  is  276.39557
W values : [[1.0574409]
 [1.0610943]
 [1.0465648]
 [1.070002 ]]
b values : [2.1176674]
Training loss at step:  15  is  274.6341
W values : [[1.2195718]
 [1.2237855]
 [1.2070284]
 [1.234059 ]]
b values : [2.4423563]
Training loss at step:  20  is  274.14352
W values : [[1.305135 ]
 [1.3096443]
 [1.2917114]
 [1.3206384]]
b values : [2.613708]
Training loss at step:  25  is  274.0069
W values : [[1.3502902]
 [1.3549556]
 [1.3364019]
 [1.36633  ]]
b values : [2.7041373]
Training loss at step:  30  is  273.9688
W values : [[1.3741204]
 [1.3788681]
 [1.3599869]
 [1.3904433]]
b values : [2.7518601]
Training loss at step:  35  is  273.95825
W values : [[1.3866965]
 [1.3914877]
 [1.3724337]
 [1.

In [0]:
sess.close()

### Get the shapes and values of W and b

In [144]:
print('shape of W : ', Weight_array.shape )
print('shape of b : ', bias_array.shape )


shape of W :  (4, 1)
shape of b :  (1,)


In [145]:
Weight_array

array([[1.4007461],
       [1.4055864],
       [1.3863372],
       [1.4173869]], dtype=float32)

In [146]:
bias_array

array([2.805182], dtype=float32)

### Model Prediction on 1st Examples in Test Dataset

In [149]:
ypred = tf.add(tf.matmul(X_test,Weight_array),bias_array,name='output')

sess1 = tf.Session()
ypred.eval(session=sess1)

array([[5.61012  ],
       [5.61028  ],
       [5.6102724],
       [5.6102104],
       [5.610093 ],
       [5.6102786],
       [5.61026  ],
       [5.6102896],
       [5.610265 ],
       [5.6102824],
       [5.6102867],
       [5.6102858],
       [5.6102962],
       [5.610273 ],
       [5.610194 ],
       [5.6102886],
       [5.610196 ],
       [5.6102557],
       [5.609956 ],
       [5.610261 ],
       [5.610283 ],
       [5.610278 ],
       [5.6101937],
       [5.61026  ],
       [5.6102185],
       [5.61022  ],
       [5.6099806],
       [5.610267 ],
       [5.609865 ],
       [5.609933 ],
       [5.610265 ],
       [5.610116 ],
       [5.610282 ],
       [5.6102037],
       [5.6102133],
       [5.610287 ],
       [5.61028  ],
       [5.6102667],
       [5.6102686],
       [5.6102657],
       [5.610289 ],
       [5.61026  ],
       [5.610273 ],
       [5.610262 ],
       [5.6102376],
       [5.610199 ],
       [5.610245 ],
       [5.6102514],
       [5.610198 ],
       [5.6102805],


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [151]:
data_I.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
X = data_I.iloc[:,1:5].values
y = data_I.iloc[:,5].values

from sklearn.preprocessing import LabelEncoder
encoder =  LabelEncoder()
y1 = encoder.fit_transform(y)

Y = pd.get_dummies(y1).values




### Splitting the data into feature set and target set

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
import numpy as np
import pandas as pd
from pandas import read_csv
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [157]:

model = Sequential()
model.add(Dense(12, input_dim=4, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

W0721 12:55:26.873008 139870843578240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0721 12:55:26.876922 139870843578240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0721 12:55:26.916698 139870843578240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0721 12:55:26.943414 139870843578240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



### Model Training 

### Model Prediction

In [159]:
ypred = model.predict(X_test)
ypred

array([[0.46154684, 0.5246377 , 0.01381546],
       [0.39297545, 0.5835152 , 0.02350934],
       [0.40914512, 0.53168684, 0.05916805],
       [0.43285817, 0.56058645, 0.00655541],
       [0.40087697, 0.5311961 , 0.06792691],
       [0.48848125, 0.5034019 , 0.00811683],
       [0.39637178, 0.5312695 , 0.07235871],
       [0.41929275, 0.5667911 , 0.01391613],
       [0.40539637, 0.58092904, 0.01367459],
       [0.41304618, 0.5654518 , 0.02150198],
       [0.4602877 , 0.5284081 , 0.0113042 ],
       [0.43185514, 0.55244356, 0.01570128],
       [0.43840697, 0.5454455 , 0.01614751],
       [0.41306707, 0.57134545, 0.01558746],
       [0.44169563, 0.54246664, 0.01583777],
       [0.39268565, 0.53554195, 0.07177238],
       [0.4391518 , 0.54340434, 0.01744387],
       [0.44938207, 0.52990097, 0.02071691],
       [0.38764217, 0.5360903 , 0.07626753],
       [0.3956174 , 0.5399035 , 0.06447913],
       [0.46687117, 0.51741946, 0.01570931],
       [0.46391255, 0.5177212 , 0.0183663 ],
       [0.

### Save the Model

In [0]:
import pickle
pickle.dump(model, open("model.pkl","wb" , ))


### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?